### Main notebook

In [1]:
# add folder for imports
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm
from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data
from pathway import Pathway
from utils import create_compound, create_reaction
# suppres rdkit warnings
import rdkit
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')


# read data from csv
cpds = pd.read_csv('../data/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('../data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('../data/pairs_final_RPAIRS_pred.csv', index_col=0)
cofactors = pd.read_csv('../data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)
pathway = Pathway()

data = create_compound(data, cpds, cofactors)
data = create_reaction(data, rxns)

# CREATE GRAPH
graph.create_graph(data=data, pairs=pairs)

graph.calculate_edge_mol_weight(data)
graph.calculate_smiles_similarity(data)

# nodes: 7997 
# edges: 11783


100%|██████████| 11783/11783 [00:52<00:00, 226.05it/s]


In [2]:
######### VALIDATION SET FROM nicepath ###########
test_cases = pd.read_csv('../data/original/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases['paths_list'] = test_cases['Pathway '].apply(lambda x: x.split(','))
print(f'Number of test cases: {len(test_cases)}')
      
from utils import validate

paths = validate(test_cases, graph, 'mol_weight')

Number of test cases: 50


 24%|██▍       | 12/50 [00:11<00:37,  1.02it/s]

***** No path found between C00078 and C07576 *****
***** Node not found for C06160 or C20299 *****


 34%|███▍      | 17/50 [00:13<00:20,  1.57it/s]

***** Path with length 2 ['C00047', 'C00408'] *****


...

***** Node not found for C00043 or C21263 *****


 52%|█████▏    | 26/50 [00:20<00:21,  1.14it/s]

***** Node not found for C20953 or C20942 *****


100%|██████████| 50/50 [01:05<00:00,  1.31s/it]

Correct pathway predictions: 23
Correct pathway predictions (%): 46.0


### Search for correct pathways in top-k predicted paths

In [3]:
import ast

def validate(test_cases: pd.DataFrame, G: Graph, method: str, i: int):
    correct_pathways = []
    paths = []
    for row in tqdm(range(len(test_cases))):
        source = test_cases['source'].iloc[row]
        target = test_cases['target'].iloc[row]
        try:
            pred_path, idx_smi, idx_com = G.constrained_shortest_path(source, target, weight=method)
            pred_path = pred_path[idx_smi[i]]
        except nx.NodeNotFound:
            print(f'***** Node not found for {source} or {target} *****')
            pred_path, idx_smi, idx_com = [], None, None
        except TypeError:
            pass
        except IndexError:
            pass
        
        correct_pathways.append((pred_path == test_cases['paths_list'].iloc[row]))
        paths.append(pred_path)
    
    print(f'Correct pathway predictions: {correct_pathways.count(True)}')
    print(f'Correct pathway predictions (%): {100 * correct_pathways.count(True) / len(correct_pathways)}')

    # return the DataFrame with the resulted pathways and correct or not
    paths = pd.DataFrame([str(p) for p in paths], columns=['Pathway'])
    paths['Pathway']  = paths['Pathway'].apply(lambda x: ast.literal_eval(x))
    paths['Correct'] = correct_pathways
    return paths

def search_for_correct_in_top_k_paths(idx, graph, i: int):
    test_cases = pd.read_csv('../data/original/test_cases.csv')
    test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
    test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
    test_cases['paths_list'] = test_cases['Pathway '].apply(lambda x: x.split(','))
    
    test_cases = test_cases.iloc[idx]
    print(f'Shape of wrong predictions: {test_cases.shape}')
    paths = validate(test_cases, graph, 'mol_weight', i)

    return paths

for i in range(1, 9):
    print(f'Search in top {i+1} paths\n')
    idx = paths[paths['Correct'] == False].index
    paths = search_for_correct_in_top_k_paths(idx, graph, i)
    print()

Search in top 2 paths

Shape of wrong predictions: (27, 4)


 11%|█         | 3/27 [00:07<01:01,  2.58s/it]

***** No path found between C00078 and C07576 *****
***** Node not found for C06160 or C20299 *****


 26%|██▌       | 7/27 [00:11<00:28,  1.43s/it]

***** Path with length 2 ['C00047', 'C00408'] *****


...

***** No path found between C00078 and C07576 *****
***** Node not found for C06160 or C20299 *****


 89%|████████▉ | 17/19 [00:13<00:01,  1.42it/s]

***** Path with length 2 ['C00047', 'C00408'] *****


100%|██████████| 19/19 [00:14<00:00,  1.31it/s]

Correct pathway predictions: 0
Correct pathway predictions (%): 0.0



In [4]:
print(f'Correct pathway predictions: {(50-19)/50 * 100}%')

Correct pathway predictions: 62.0%
